# __TelecomX - Parte 2: Modelos de prediccion__

## __Objetivo__

La empresa busca anticiparse al problema de cancelacion de clientes, por lo que vamos a construir un pipeline robusto para la etapa inicial del modelado cuyas tareas son lo siguiente:

- Preparar los datos (limpieza, codificacion y normalizacion).
- Realizar analisis de correlacion y seleccion de variables.
- Entrenar diferentes modelos de clasificacion, evaluar el rendimiento a traves de distintas metricas e interpretar los resultados de cada uno.
- Crear un reporte con algunas recomendaciones sobre los principales factores que influyen en la cancelacion de clientes.

## __Importar Librerias__

In [3]:
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap

import warnings
#import requests
import pickle

from timeit import default_timer as timer
from datetime import datetime

import imblearn
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import SMOTENC

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, cross_validate, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_auc_score, RocCurveDisplay, precision_recall_curve
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler, StandardScaler, RobustScaler
from sklearn.compose import make_column_transformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.svm import SVC
from sklearn.inspection import permutation_importance

import xgboost
from xgboost import XGBClassifier

In [4]:
# version de las libreria utilizadas
print(f'numpy=={np.__version__}')
print(f'pandas=={pd.__version__}')
print(f'matplotlib=={matplotlib.__version__}')
print(f'seaborn=={sns.__version__}')
print(f'scikit-learn=={sklearn.__version__}')
print(f'imblearn=={sklearn.__version__}')
print(f'xgboost=={xgboost.__version__}')

numpy==2.2.6
pandas==2.3.1
matplotlib==3.10.3
seaborn==0.13.2
scikit-learn==1.6.1
imblearn==1.6.1
xgboost==3.0.5


## __Preprocesamiento de los datos__

In [5]:
# cargar datos preprocesados desde GitHub
data = pd.read_csv('https://raw.githubusercontent.com/jbernalg/challenge_TelecomX/refs/heads/main/data_out/datos_tratados.csv', sep=',')
data.head()

,cancelacion,genero,jubilado,en_pareja,dependientes,meses_contrato,servicio_telefonico,multiples_lineas,servicio_internet,seguridad_online,...,proteccion_dispositivo,soporte_efectivo,streaming_TV,streaming_peliculas,tipo_contrato,facturacion_electronica,metodo_pago,cuenta_mensual,cuentas_total,cuentas_diarias
0,0,Femenino,No,Si,Si,9,Si,No,DSL,No,...,No,Si,Si,No,un year,Si,cheque por correo,65.6,593.30,0.242963
1,0,Masculino,No,No,No,9,Si,Si,DSL,No,...,No,No,No,Si,mes a mes,No,cheque por correo,59.9,542.40,0.221852
2,1,Masculino,No,No,No,4,Si,No,Fibra_optica,No,...,Si,No,No,No,mes a mes,Si,cheque electronico,73.9,280.85,0.615833
3,1,Masculino,Si,Si,No,13,Si,No,Fibra_optica,No,...,Si,No,Si,Si,mes a mes,Si,cheque electronico,98.0,1237.85,0.251282
4,1,Femenino,Si,Si,No,3,Si,No,Fibra_optica,No,...,No,Si,Si,No,mes a mes,Si,cheque por correo,83.9,267.40,0.932222


> Vamos a eliminar la variable _cuentas_diarias_ debido a que fue obtenida a partir de _cuenta_mensual_ por lo que no aporta informacion relevante.

In [6]:
datos = data.drop(['cuentas_diarias'], axis=1)
datos.head()

,cancelacion,genero,jubilado,en_pareja,dependientes,meses_contrato,servicio_telefonico,multiples_lineas,servicio_internet,seguridad_online,respaldo_online,proteccion_dispositivo,soporte_efectivo,streaming_TV,streaming_peliculas,tipo_contrato,facturacion_electronica,metodo_pago,cuenta_mensual,cuentas_total
0,0,Femenino,No,Si,Si,9,Si,No,DSL,No,Si,No,Si,Si,No,un year,Si,cheque por correo,65.6,593.30
1,0,Masculino,No,No,No,9,Si,Si,DSL,No,No,No,No,No,Si,mes a mes,No,cheque por correo,59.9,542.40
2,1,Masculino,No,No,No,4,Si,No,Fibra_optica,No,No,Si,No,No,No,mes a mes,Si,cheque electronico,73.9,280.85
3,1,Masculino,Si,Si,No,13,Si,No,Fibra_optica,No,Si,Si,No,Si,Si,mes a mes,Si,cheque electronico,98.0,1237.85
4,1,Femenino,Si,Si,No,3,Si,No,Fibra_optica,No,No,No,Si,Si,No,mes a mes,Si,cheque por correo,83.9,267.40


In [ ]:
# resumen estadistico de las variables numericas
datos.describe()

,cancelacion,meses_contrato,cuenta_mensual,cuentas_total
count,7032.000000,7032.000000,7032.000000,7032.000000
mean,0.265785,32.421786,64.798208,2283.300441
std,0.441782,24.545260,30.085974,2266.771362
min,0.000000,1.000000,18.250000,18.800000
25%,0.000000,9.000000,35.587500,401.450000
50%,0.000000,29.000000,70.350000,1397.475000
75%,1.000000,55.000000,89.862500,3794.737500
max,1.000000,72.000000,118.750000,8684.800000
